<a href="https://colab.research.google.com/github/towardsai/cours-demo-notebooks/blob/main/exercice_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installation des bibliothèques nécessaires
!pip install instructor pydantic google-generativeai jsonref

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 1.8 MB/s eta 0:00:00


In [ ]:
# Importation des bibliothèques
import instructor
import google.generativeai as genai
from pydantic import BaseModel, Field
from enum import Enum
from typing import Literal

In [ ]:
# Configuration de l'API Gemini
GOOGLE_API_KEY = ""  # Remplacez par votre clé API
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
# 1. Définition des modèles Pydantic
class Personne(BaseModel):
    nom: str = Field(max_length=50)
    age: int | str
    role: str = Field(
        description="Role de la personne dans la compagnie, ex: Développeur",
    )
    compagnie: Literal["NeuroTech Systems", "VirtuaSoft", "Quantum Solutions", "OTHER"]

class ExtractionPersonnes(BaseModel):
    reasoning: str = Field(
        description="À partir des informations fournies, écris 'comment' tu vas t'y prendre pour extraire les informations du texte.",
    )
    list_personnes: list[Personne] = Field(min_length=3)

In [ ]:
# 2. Initialisation du client Gemini avec JSON STRICT pour l'extraction SEULEMENT
client = instructor.from_gemini(
    client=genai.GenerativeModel(model_name="models/gemini-2.0-flash-lite"),
    mode=instructor.Mode.GEMINI_JSON  # Forcer JSON uniquement pour l'extraction
)

In [ ]:
# 3. Texte d'entrée
texte = """
Thomas, 34 ans, est ingénieur en intelligence artificielle chez NeuroTech Systems, une startup spécialisée dans les algorithmes d’apprentissage profond.
Il passe ses journées à entraîner des modèles pour optimiser la reconnaissance vocale, souvent en collaboration avec Léa, 29 ans, designer UX chez VirtuaSoft,
une entreprise qui développe des interfaces immersives pour la réalité augmentée.
Léa s’assure que les utilisateurs interagissent intuitivement avec les IA conçues par des entreprises comme celle de Thomas.
De son côté, Karim, 41 ans, est chef de projet chez Quantum Solutions, un cabinet de conseil en innovation technologique.
Il supervise des partenariats entre sociétés comme NeuroTech et VirtuaSoft, coordonnant leurs efforts pour créer des expériences utilisateur toujours plus fluides et intelligentes.
"""

In [ ]:
# 4. Préparer le message pour extraire les personnes
messages = [
    {"role": "user", "content": f"Extrais toutes les personnes mentionnées dans ce texte: {texte}"}
]

In [ ]:
# 5. Extraction des personnes sous forme de générateur
extraction_personnes = client.chat.completions.create(
    messages=messages,
    response_model=ExtractionPersonnes,  # Attente d'un objet contenant "personnes"
    max_retries=3,
)

In [ ]:
# 7. Affichage du raisonnement et des noms et rôles des personnes extraites

print("Raisonnement du modèle: \n")
print(extraction_personnes.reasoning)


print("\nListe des personnes extraites :\n")
for i, personne in enumerate(extraction_personnes.list_personnes, start=1):
    print(f"Personne {i}:\nNom: {personne.nom}, Rôle: {personne.role}, Compagnie: {personne.compagnie}, Age: {personne.age}")
    print("Ou, directement: "+str(personne)+"\n")


Raisonnement du modèle: 

Je vais parcourir le texte pour identifier les noms de personnes, leurs âges, leurs rôles et les entreprises pour lesquelles ils travaillent. Je vais utiliser les informations fournies pour remplir les champs correspondants dans le schéma JSON.

Liste des personnes extraites :

Personne 1:
Nom: Thomas, Rôle: ingénieur en intelligence artificielle, Compagnie: NeuroTech Systems, Age: 34
Ou, directement: nom='Thomas' age=34 role='ingénieur en intelligence artificielle' compagnie='NeuroTech Systems'

Personne 2:
Nom: Léa, Rôle: designer UX, Compagnie: VirtuaSoft, Age: 29
Ou, directement: nom='Léa' age=29 role='designer UX' compagnie='VirtuaSoft'

Personne 3:
Nom: Karim, Rôle: chef de projet, Compagnie: Quantum Solutions, Age: 41
Ou, directement: nom='Karim' age=41 role='chef de projet' compagnie='Quantum Solutions'



In [ ]:
# 8. Ajouter les informations extraites au contexte
personnes_info = "\n".join([f"{p.nom}, {p.age} ans, {p.role} chez {p.compagnie}" for p in extraction_personnes.list_personnes])
messages.append({"role": "assistant", "content": personnes_info})

In [ ]:
# 9. Ajouter une nouvelle question pour analyser les relations
messages.append({
    "role": "user",
    "content": "Quelle est la relation entre chacune de ces personnes ?"
})

In [ ]:
# ** Désactiver le mode JSON pour la réponse textuelle**
gemini_client_text = instructor.from_gemini(
    client=genai.GenerativeModel(model_name="models/gemini-2.0-flash-lite")
)

# 10. Corriger l'erreur en ajoutant `response_model=str`
reponse_2 = gemini_client_text.chat.completions.create(
    messages=messages,
    response_model=str  # On attend une réponse textuelle libre
)

In [ ]:
# 11. Afficher la réponse finale
print("Analyse des relations :")
print(reponse_2)

Analyse des relations :
Thomas collabore avec Léa. Karim supervise des partenariats entre les entreprises où travaillent Thomas et Léa
